In [12]:
import tensorflow as tf
from tensorflow import keras
from keras import layers, models
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Activation, Conv2, Dense, Dropout
from tensorflow.keras.optimizers import Adagrad, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import tensorflow_hub as hub

import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy

from PIL import Image


# データを正方形にリサイズするときの辺の長さ
square_length = 64

# 画像を置いてるパス
image_path = "../image/eye_dlib"

# ラベル付け(0春 1夏 2冬)
# TODO: ? フォルダ名に数字入れておくとかのほうがいいかもしれない？
# TODO: 3つ以上に増やす場合は、class_modeやlossをbinaryからcategricalに変える
def get_label(value):
    if value == "Ariel":
        return 0
    elif value == "Other":
        return 1
    
# ラベル更新したら一緒に更新する
def max_label():
    return 2

In [13]:
# 画像の読み込み
datagen = ImageDataGenerator(
    rescale=1./255
#     validation_split=0.1
    )

train_generator = datagen.flow_from_directory(
    image_path + "/train",
    target_size=(square_length, square_length),
    class_mode='binary',
    batch_size=32,
    subset='training',
)

Found 17206 images belonging to 2 classes.


In [18]:
print("start")

# モデルの構築
model = keras.Sequential([
#     keras.layers.Conv2D(32, (8,8), activation="relu"),
#     keras.layers.Conv2D(32, (8,8), activation="relu"),
#     keras.layers.MaxPooling2D(pool_size=(4,4)),
#     keras.layers.Flatten(),
#     keras.layers.Dropout(0.2),
#     keras.layers.Dense(200, activation="relu"),
#     keras.layers.Dropout(0.4),
#     keras.layers.Dense(200, activation="relu"),
#     keras.layers.Dropout(0.4),
#     keras.layers.Dense(1),
#     keras.layers.Activation("softmax")
    hub.KerasLayer(
        "https://tfhub.dev/tensorflow/efficientnet/b4/feature-vector/1",
        trainable=False,
    ),
    keras.layers.Dense(1, activation="sigmoid")
])
# memo: shapeで警告出るのは、keras.layers.reshape((...)),imput_shape=(...))を使えば良さそう

# # モデルをコンパイル
model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=0.001), metrics=["accuracy"])
# 学習を実行
model.fit(train_generator, steps_per_epoch=1, epochs=30)
print(model.summary())

start
Epoch 1/30
1/1 [==============================] - 18s 18s/step - loss: 0.7293 - accuracy: 0.4062
Epoch 2/30
1/1 [==============================] - 1s 678ms/step - loss: 0.6657 - accuracy: 0.5938
Epoch 3/30
1/1 [==============================] - 1s 827ms/step - loss: 0.6735 - accuracy: 0.5625
Epoch 4/30
1/1 [==============================] - 1s 601ms/step - loss: 0.6912 - accuracy: 0.5625
Epoch 5/30
1/1 [==============================] - 1s 558ms/step - loss: 0.6925 - accuracy: 0.5000
Epoch 6/30
1/1 [==============================] - 1s 572ms/step - loss: 0.7028 - accuracy: 0.4062
Epoch 7/30
1/1 [==============================] - 1s 579ms/step - loss: 0.7024 - accuracy: 0.4062
Epoch 8/30
1/1 [==============================] - 1s 566ms/step - loss: 0.6959 - accuracy: 0.4688
Epoch 9/30
1/1 [==============================] - 1s 631ms/step - loss: 0.6475 - accuracy: 0.6562
Epoch 10/30
1/1 [==============================] - 1s 619ms/step - loss: 0.6570 - accuracy: 0.5938
Epoch 11/30
1/

In [21]:
# テスト用ディレクトリの画像でチェック。正解率を表示する。
total = 0.
ok_count = 0.

test_path = image_path + "/test"
for dir in os.listdir(test_path):
    if dir == ".DS_Store":
        continue

    dir_path = test_path + "/" + dir
    label = get_label(dir)

    for file in os.listdir(dir_path):
        if file != ".DS_Store":
            file_path = dir_path + "/" + file
            image = np.array(Image.open(file_path).resize(size=(square_length, square_length)))
#             image = image.transpose(2, 0, 1)
#             image = image.reshape(image.shape[0] * image.shape[1] * image.shape[2], 1)
            result = model.predict(np.array([image / 255.]))
            print("[" + str(int(total)) + "]label:", label, "result:", result)

            total += 1.

            if abs(label - result) < 0.5:
                ok_count += 1.

print("seikai: ", ok_count / total * 100, "%")

[0]label: 0 result: [[0.44059673]]
[1]label: 0 result: [[0.6008303]]
[2]label: 0 result: [[0.44190586]]
[3]label: 0 result: [[0.3900989]]
[4]label: 0 result: [[0.39919227]]
[5]label: 0 result: [[0.39628744]]
[6]label: 0 result: [[0.366643]]
[7]label: 0 result: [[0.4047255]]
[8]label: 0 result: [[0.3978133]]
[9]label: 0 result: [[0.506678]]
[10]label: 0 result: [[0.35928857]]
[11]label: 0 result: [[0.42420653]]
[12]label: 0 result: [[0.37527177]]
[13]label: 0 result: [[0.43883997]]
[14]label: 0 result: [[0.5089568]]
[15]label: 0 result: [[0.3924995]]
[16]label: 1 result: [[0.5698696]]
[17]label: 1 result: [[0.6470584]]
[18]label: 1 result: [[0.6219007]]
[19]label: 1 result: [[0.5589263]]
[20]label: 1 result: [[0.4711591]]
[21]label: 1 result: [[0.66509145]]
[22]label: 1 result: [[0.49874684]]
[23]label: 1 result: [[0.54446924]]
[24]label: 1 result: [[0.5767686]]
[25]label: 1 result: [[0.5646341]]
[26]label: 1 result: [[0.46717182]]
[27]label: 1 result: [[0.5831383]]
[28]label: 1 result:

In [19]:
model.save(image_path + '/model')

INFO:tensorflow:Assets written to: ../image/eye_dlib/model\assets


INFO:tensorflow:Assets written to: ../image/eye_dlib/model\assets


end


In [20]:
model.save(image_path + "/model.h5")